<a href="https://colab.research.google.com/github/skondaparthi/FinSent-Agent/blob/main/FinSent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from google.colab import userdata

PAT = userdata.get('githubPAT')

In [ ]:
!git clone https://{PAT}@github.com/skondaparthi/FinSent-Agent.git

%cd FinSent-Agent

from ingest.news_api import find_news
import models.finbert_model as finbert_model

username = userdata.get("gitUsername")
email = userdata.get("gitEmail")

%shell git config --global user.name "{username}"
%shell git config --global user.email "{email}"


In [7]:
from importlib import reload

# Import the module first
import ingest.news_api

# Now you can reload it
reload(ingest.news_api)

from ingest.news_api import find_news

In [62]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ingest/__pycache__/news_api.cpython-311.pyc
	modified:   ingest/news_api.py
	modified:   models/__pycache__/finbert_model.cpython-311.pyc

no changes added to commit (use "git add" and/or "git commit -a")


In [63]:
!git add .

In [64]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   ingest/__pycache__/news_api.cpython-311.pyc
	modified:   ingest/news_api.py
	modified:   models/__pycache__/finbert_model.cpython-311.pyc



In [69]:
!git commit -m "added embeddings to df"

[main 1d20c3f] added embeddings to df
 3 files changed, 1 insertion(+), 1 deletion(-)


In [70]:
!git push https://{PAT}@github.com/skondaparthi/FinSent-Agent.git main

Enumerating objects: 16, done.
Counting objects: 100% (16/16), done.
Delta compression using up to 2 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 1.95 KiB | 1.95 MiB/s, done.
Total 9 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/skondaparthi/FinSent-Agent.git
   6d9fc74..1d20c3f  main -> main


In [8]:
news_df = find_news()

display(news_df.head())

,source,title,description,publishedAt
0,The Verge,Zuckerberg’s ‘personal superintelligence’ plan...,It has been another busy week. GPT-5 appears t...,2025-08-02T00:11:36Z
1,Gizmodo.com,Wall Street’s AI Bubble Is Worse Than the 1999...,A chief economist at investment giant Apollo s...,2025-07-17T13:00:10Z
2,Gizmodo.com,Truth Social Needs Its Own Grok. Trump Has the...,A MAGA-bot could be on the way.,2025-07-17T13:15:25Z
3,Gizmodo.com,"As Tesla’s Sales Dwindle, a Critical Revenue S...",Tesla needs all the help it can get.,2025-07-23T13:10:27Z
4,Gizmodo.com,Studies Show AI Models Love to Share With One ...,Sharing is caring.,2025-07-30T18:10:01Z


In [9]:
news_df = finbert_model.score_df(news_df)
news_df.head()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,source,title,description,publishedAt,sent_scores,category
0,The Verge,Zuckerberg’s ‘personal superintelligence’ plan...,It has been another busy week. GPT-5 appears t...,2025-08-02T00:11:36Z,"{'negative': 0.061627395, 'neutral': 0.0202904...",positive
1,Gizmodo.com,Wall Street’s AI Bubble Is Worse Than the 1999...,A chief economist at investment giant Apollo s...,2025-07-17T13:00:10Z,"{'negative': 0.025278883, 'neutral': 0.7703285...",neutral
2,Gizmodo.com,Truth Social Needs Its Own Grok. Trump Has the...,A MAGA-bot could be on the way.,2025-07-17T13:15:25Z,"{'negative': 0.06825258, 'neutral': 0.04631966...",positive
3,Gizmodo.com,"As Tesla’s Sales Dwindle, a Critical Revenue S...",Tesla needs all the help it can get.,2025-07-23T13:10:27Z,"{'negative': 0.23368655, 'neutral': 0.01441254...",positive
4,Gizmodo.com,Studies Show AI Models Love to Share With One ...,Sharing is caring.,2025-07-30T18:10:01Z,"{'negative': 0.08102807, 'neutral': 0.02253392...",positive


In [61]:
news_df["embeds"] = finbert_model.embed((news_df["title"] + " " +
                                        news_df["description"]).tolist()).tolist()
news_df.head()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,source,title,description,publishedAt,sent_scores,category,embeds
0,The Verge,Zuckerberg’s ‘personal superintelligence’ plan...,It has been another busy week. GPT-5 appears t...,2025-08-02T00:11:36Z,"{'negative': 0.061627395, 'neutral': 0.0202904...",positive,"[-0.0854862853884697, -0.029945576563477516, 0..."
1,Gizmodo.com,Wall Street’s AI Bubble Is Worse Than the 1999...,A chief economist at investment giant Apollo s...,2025-07-17T13:00:10Z,"{'negative': 0.025278883, 'neutral': 0.7703285...",neutral,"[0.0006281165988184512, -0.07895796000957489, ..."
2,Gizmodo.com,Truth Social Needs Its Own Grok. Trump Has the...,A MAGA-bot could be on the way.,2025-07-17T13:15:25Z,"{'negative': 0.06825258, 'neutral': 0.04631966...",positive,"[-0.0829867348074913, -0.03853766992688179, -0..."
3,Gizmodo.com,"As Tesla’s Sales Dwindle, a Critical Revenue S...",Tesla needs all the help it can get.,2025-07-23T13:10:27Z,"{'negative': 0.23368655, 'neutral': 0.01441254...",positive,"[-0.04854635149240494, 0.03438419848680496, 0...."
4,Gizmodo.com,Studies Show AI Models Love to Share With One ...,Sharing is caring.,2025-07-30T18:10:01Z,"{'negative': 0.08102807, 'neutral': 0.02253392...",positive,"[-0.05139394477009773, -0.043352656066417694, ..."
